# This Python notbook generates interactive 2D velocity fields from 01EMMA_CYL experiments.

Data are collected from the LEGI opendap server : **http://servdap.legi.grenoble-inp.fr/opendap/coriolis/01EMMA_CYL/**

Plotting, data retrieving and interactive python functions used in this notebook are available in the file *func_01emma_cyl.py* in the root directory

In [ ]:
import func_01emma_cyl
func_01emma_cyl.create_plot()